In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 16 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

06 February 2023 ; 16:36:31


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,24,0,9.1,1.5,3.5,0.440,...,2.42,113.98,111.67,2.32,2.23,113.97,111.82,2.16,1,1
11,MarJon Beauchamp,SF,22,MIL,34,9,14.9,2.1,5.4,0.399,...,2.42,113.98,111.67,2.32,2.23,113.97,111.82,2.16,1,1
17,A.J. Lawson,SG,22,DAL,11,0,3.5,0.7,1.5,0.500,...,0.35,116.67,116.34,0.33,0.44,116.67,116.27,0.40,1,1
25,Jaden Hardy,SG,20,DAL,25,0,10.1,2.0,5.0,0.395,...,0.35,116.67,116.34,0.33,0.44,116.67,116.27,0.40,1,1
36,AJ Griffin,SF,19,ATL,50,11,20.8,3.8,7.8,0.487,...,-0.24,115.43,115.64,-0.22,-0.36,115.49,115.82,-0.34,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,33.7,ORL,15.6,0.523404,2.965957,6.6,20.3,13,0.282726
1,Bennedict Mathurin,28.4,IND,13.5,-0.745455,2.385455,5.5,17.4,11,0.150979
2,Walker Kessler,20.3,UTA,21.1,1.929412,6.913725,3.4,7.8,10,0.121210
3,Jaden Ivey,30.1,DET,12.2,-0.656000,0.164000,5.2,15.2,15,0.064190
4,Jalen Duren,24.8,DET,16.6,0.534783,5.347826,3.5,8.3,15,0.059096


In [7]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)